In [1]:
%cd capstone-project-9900w12achatllm

/gemini/code/capstone-project-9900w12achatllm


In [2]:
!pip install -e ".[torch,metrics]"
!pip install --upgrade pandas pyarrow datasets
!pip install auto_gptq>=0.5.0
!pip install optimum
!pip uninstall apex -y

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///gemini/code/capstone-project-9900w12achatllm
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.

batch size = 3, max_example = 500, lr = 1e-4, epoch=2

In [3]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 1e-04 \
    --num_train_epochs 2.0 \
    --max_samples 500 \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-25_epoch=2 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 10 \
    --per_device_eval_batch_size 2

[2024-07-11 12:23:23,556] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 12:23:42 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:23:42,167 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:23:42,167 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:23:42,167 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:23:42,167 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:23:42,167 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:23:42,167 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 12:23:42,554 >> Special tokens have been added in the vocabulary, 

In [4]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/eval_2024-07-09-00-26-34_epoch=2 \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-25_epoch=2

[2024-07-11 12:34:31,725] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 12:34:51 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:34:51,534 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:34:51,534 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:34:51,534 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:34:51,534 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:34:51,534 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:34:51,534 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 12:34:51,978 >> Special tokens have been added in the vocabulary, make sure

batch size = 3, max_exmaple = 500, lr=1e-4, epoch=5

In [5]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 1e-04 \
    --num_train_epochs 5.0 \
    --max_samples 500 \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-26_epoch=5 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 10 \
    --per_device_eval_batch_size 2

[2024-07-11 12:38:38,322] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 12:39:00 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:39:00,067 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:39:00,067 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:39:00,067 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:39:00,067 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:39:00,067 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 12:39:00,067 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 12:39:00,423 >> Special tokens have been added in the vocabulary, 

In [6]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/eval_2024-07-09-00-26-35_epoch=5 \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-26_epoch=5

[2024-07-11 13:01:29,646] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 13:01:50 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:01:50,854 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:01:50,854 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:01:50,854 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:01:50,854 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:01:50,854 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:01:50,854 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 13:01:51,171 >> Special tokens have been added in the vocabulary, make sure

batch size = 3, max_example = 500, lr=1e-4, epoch=7

In [7]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 1e-04 \
    --num_train_epochs 7.0 \
    --max_samples 500 \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-27_epoch=7 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 10 \
    --per_device_eval_batch_size 2

[2024-07-11 13:05:50,601] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 13:06:08 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:06:08,732 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:06:08,733 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:06:08,733 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:06:08,733 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:06:08,733 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:06:08,733 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 13:06:09,045 >> Special tokens have been added in the vocabulary, 

In [8]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/eval_2024-07-09-00-26-36_epoch=7 \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-27_epoch=7

[2024-07-11 13:36:17,240] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 13:36:37 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:36:37,845 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:36:37,845 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:36:37,845 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:36:37,845 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:36:37,845 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 13:36:37,846 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 13:36:38,191 >> Special tokens have been added in the vocabulary, make sure